In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
import os

In [ ]:
#Set execution flags
RUNNING_ON_FLOYDHUB = False
epochs=1

In [ ]:
# Local data directories
if not RUNNING_ON_FLOYDHUB:
    print('Linking to local data directories.')
    train_dir = 'data/dogs-vs-cats-small/train'
    test_dir = 'data/dogs-vs-cats-small/test'
    validation_dir = 'data/dogs-vs-cats-small/validation'
    output_dir = 'models/'
else:
    print('Linking to floydhub data directories mounted at /data .')
    train_dir = '/data/dogs-vs-cats-small/train'
    test_dir = '/data/dogs-vs-cats-small/test'
    validation_dir = '/data/dogs-vs-cats-small/validation'
    output_dir = '/output'

### Image Augmentation

In [ ]:
datagen = image.ImageDataGenerator(rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  fill_mode='nearest',
                                  horizontal_flip=True,
                                  data_format='channels_last')

In [ ]:
# Filenames from training directory
file_idx = 100
train_cat_files = [ os.path.join(train_dir, 'cats', fname) for fname in os.listdir(os.path.join( train_dir, 'cats')) ]
img = image.load_img(train_cat_files[file_idx], target_size=(150, 150))
img_array = image.img_to_array(img)
print('Array shape: {0}.'.format(img_array.shape))

print('ImageDataGenerator.flow() needs shape to be in format (samples, height, width, channel).')
img_array = img_array.reshape((1,) + img_array.shape)
print('New array shape: {0}.'.format(img_array.shape))

In [ ]:
plt.figure()
plt.imshow(image.array_to_img(img_array[0]))
plt.title('Original image')
plt.show()

In [ ]:
count = 1
fig = plt.figure()
for augmented_batch in datagen.flow(img_array, batch_size=1):
    ax = fig.add_subplot(2, 2, count)
    ax.imshow(image.array_to_img(augmented_batch[0]))
    count += 1
    if count % 5 == 0:
        break
plt.suptitle('Augmented images')
plt.show()

In [ ]:
test_datagen = image.ImageDataGenerator(rescale=1./255) #Should not modify test and validation images
train_datagen = image.ImageDataGenerator(rotation_range=40,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2,
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         horizontal_flip=True,
                                         rescale=1./255)

In [ ]:
train_batches = train_datagen.flow_from_directory(train_dir,
                                                target_size=(150, 150),
                                                class_mode='binary',
                                                batch_size=20)
validation_batches = test_datagen.flow_from_directory(validation_dir,
                                                   target_size=(150, 150),
                                                   class_mode='binary',
                                                   batch_size=20)

### Model

In [ ]:
from keras import models, layers, optimizers

In [ ]:
#Create convolutional neural network
convnet = models.Sequential()

convnet.add( layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)) )
convnet.add( layers.MaxPooling2D() )

convnet.add( layers.Conv2D(64, (3, 3), activation='relu') )
convnet.add( layers.MaxPooling2D() )

convnet.add( layers.Conv2D(128, (3, 3), activation='relu') )
convnet.add( layers.MaxPooling2D() )

convnet.add( layers.Conv2D(128, (3, 3), activation='relu') )
convnet.add( layers.MaxPooling2D() )

convnet.add( layers.Flatten() )
convnet.add( layers.Dropout(0.5) )
convnet.add( layers.Dense( 512, activation='relu' ) )
convnet.add( layers.Dense( 1, activation='sigmoid' ) )

convnet.summary()

In [ ]:
convnet.compile( optimizer=optimizers.RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'] )

### Train

In [ ]:
history = convnet.fit_generator(train_batches, steps_per_epoch=100, epochs=epochs, validation_data=validation_batches, validation_steps=50)
convnet.save(os.path.join(output_dir, 'cats_vs_dogs_small_dropout_2.h5'))

### Plots

In [ ]:
val_loss = history.history['val_loss']
val_acc = history.history['val_acc']

train_loss = history.history['loss']
train_acc = history.history['acc']

epoch = history.epoch

In [ ]:
# Loss plots
plt.figure()

plt.plot(epoch, train_loss, 'bo', label='Training loss')
plt.plot(epoch, val_loss, 'rx', label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy plots
plt.figure()

plt.plot( epoch, train_acc, 'bo', label='Training accuracy' )
plt.plot( epoch, val_acc, 'rx', label='Validation accuracy' )
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()